<a href="https://colab.research.google.com/github/Mayank735254/ML-task/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
os.listdir()

['.config',
 'description.txt',
 'test_data.txt',
 'test_data_solution.txt',
 'train_data.txt',
 'sample_data']

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
data=pd.read_csv('/content/description.txt')
data


,Train data:
0,ID ::: TITLE ::: GENRE ::: DESCRIPTION
1,ID ::: TITLE ::: GENRE ::: DESCRIPTION
2,ID ::: TITLE ::: GENRE ::: DESCRIPTION
3,ID ::: TITLE ::: GENRE ::: DESCRIPTION
4,Test data:
5,ID ::: TITLE ::: DESCRIPTION
6,ID ::: TITLE ::: DESCRIPTION
7,ID ::: TITLE ::: DESCRIPTION
8,ID ::: TITLE ::: DESCRIPTION
9,Source:


In [4]:
def load_data(file_path):
  with open(file_path,'r',encoding='utf-8') as f:
    data=f.readlines()
  data=[line.strip().split(' ::: ') for line in data ]
  return data


In [5]:
train_data=load_data('/content/train_data.txt')
train_df=pd.DataFrame(train_data,columns=['ID','Title','Genre','Description'])

test_data=load_data('/content/test_data.txt')
test_df=pd.DataFrame(test_data,columns=['ID','Title','Description'])

test_solution=load_data('/content/test_data_solution.txt')
test_solution_df=pd.DataFrame(test_solution,columns=['ID','Title','Genre','Description'])


In [6]:
print("Train Data:")
train_df



Train Data:


,ID,Title,Genre,Description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on B...
54210,54211,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The siste...
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about gr..."
54212,54213,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and have...


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(max_features=5000)


X_train_tfidf=vectorizer.fit_transform(train_df['Description'])
X_test_tfidf=vectorizer.transform(test_df['Description'])


print(f"Training data shape: {X_train_tfidf.shape}")
print(f"Test data shape: {X_test_tfidf.shape}")



Training data shape: (54214, 5000)
Test data shape: (54200, 5000)


In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_train=label_encoder.fit_transform(train_df['Genre'])
print(f"Unique genres in the training data: {label_encoder.classes_}")


Unique genres in the training data: ['action' 'adult' 'adventure' 'animation' 'biography' 'comedy' 'crime'
 'documentary' 'drama' 'family' 'fantasy' 'game-show' 'history' 'horror'
 'music' 'musical' 'mystery' 'news' 'reality-tv' 'romance' 'sci-fi'
 'short' 'sport' 'talk-show' 'thriller' 'war' 'western']


In [13]:
from sklearn.linear_model import LogisticRegression
linear_model=LogisticRegression(max_iter=500)
linear_model.fit(X_train_tfidf,y_train)

y_pred=linear_model.predict(X_test_tfidf)
Predicted_genres = label_encoder.inverse_transform(y_pred)

test_df['Predicted_Genre'] = Predicted_genres
test_df[['Title','Predicted_Genre']]



,Title,Predicted_Genre
0,Edgar's Lunch (1998),drama
1,La guerra de papá (1977),drama
2,Off the Beaten Track (2010),documentary
3,Meu Amigo Hindu (2015),drama
4,Er nu zhai (1955),drama
...,...,...
54195,"""Tales of Light & Dark"" (2013)",drama
54196,Der letzte Mohikaner (1965),action
54197,Oliver Twink (2007),comedy
54198,Slipstream (1973),drama


In [15]:
test_df['Predicted_Genre'] = Predicted_genres

merged_df = pd.merge(test_solution_df[['ID', 'Genre']], test_df[['ID','Predicted_Genre']], on='ID')
merged_df


,ID,Genre,Predicted_Genre
0,1,thriller,drama
1,2,comedy,drama
2,3,documentary,documentary
3,4,drama,drama
4,5,drama,drama
...,...,...,...
54195,54196,horror,drama
54196,54197,western,action
54197,54198,adult,comedy
54198,54199,drama,drama


In [16]:
from sklearn.metrics import accuracy_score,classification_report
accuracy=accuracy_score(merged_df['Genre'],merged_df['Predicted_Genre'])
print(f"Accuracy: {accuracy:4f}")


print("\nClassification Report:")
print(classification_report(merged_df['Genre'],merged_df['Predicted_Genre']))



Accuracy: 0.588690

Classification Report:
              precision    recall  f1-score   support

      action       0.47      0.29      0.36      1314
       adult       0.63      0.25      0.36       590
   adventure       0.58      0.15      0.24       775
   animation       0.50      0.05      0.09       498
   biography       0.00      0.00      0.00       264
      comedy       0.53      0.59      0.56      7446
       crime       0.35      0.03      0.05       505
 documentary       0.68      0.86      0.76     13096
       drama       0.55      0.77      0.64     13612
      family       0.48      0.09      0.15       783
     fantasy       0.59      0.03      0.06       322
   game-show       0.89      0.49      0.63       193
     history       0.00      0.00      0.00       243
      horror       0.64      0.56      0.60      2204
       music       0.65      0.47      0.55       731
     musical       0.36      0.02      0.03       276
     mystery       0.40      0.01     

In [17]:
from sklearn.naive_bayes import MultinomialNB
nb_model=MultinomialNB()
nb_model.fit(X_train_tfidf,y_train)



MultinomialNB()

In [18]:
y_pred_nb=nb_model.predict(X_test_tfidf)
Predicted_genres_nb = label_encoder.inverse_transform(y_pred_nb)


test_df['Predicted_Genre_NB'] = Predicted_genres_nb
merged_df_nb = pd.merge(test_solution_df,test_df[['ID','Predicted_Genre_NB']], on = 'ID')



In [19]:
from sklearn.metrics import accuracy_score,classification_report

accuracy_nb=accuracy_score(merged_df_nb['Genre'],merged_df_nb['Predicted_Genre_NB'])
print(f"Naive Bayes Accuracy : {accuracy_nb}")


print("Naive Bayes Classification Report:")
print(classification_report(merged_df_nb['Genre'],merged_df_nb['Predicted_Genre_NB']))


Naive Bayes Accuracy : 0.5138376383763837
Naive Bayes Classification Report:
              precision    recall  f1-score   support

      action       0.61      0.07      0.13      1314
       adult       0.53      0.04      0.07       590
   adventure       0.78      0.04      0.08       775
   animation       0.00      0.00      0.00       498
   biography       0.00      0.00      0.00       264
      comedy       0.52      0.40      0.45      7446
       crime       0.00      0.00      0.00       505
 documentary       0.56      0.88      0.69     13096
       drama       0.45      0.83      0.58     13612
      family       1.00      0.00      0.00       783
     fantasy       0.00      0.00      0.00       322
   game-show       0.96      0.12      0.22       193
     history       0.00      0.00      0.00       243
      horror       0.73      0.28      0.41      2204
       music       0.76      0.10      0.18       731
     musical       0.00      0.00      0.00       276
    

In [20]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train_tfidf, y_train)


SVC(kernel='linear')

In [21]:
y_pred_svm = svm_model.predict(X_test_tfidf)
Predicted_genres_svm = label_encoder.inverse_transform(y_pred_svm)


test_df['Predicted_Genre_SVM'] = Predicted_genres_svm
merged_df_svm = pd.merge(test_solution_df, test_df[['ID', 'Predicted_Genre_SVM']], on='ID')



In [22]:
from sklearn.metrics import accuracy_score, classification_report

accuracy_svm = accuracy_score(merged_df_svm['Genre'], merged_df_svm['Predicted_Genre_SVM'])
print(f"SVM Accuracy: {accuracy_svm}")


print("SVM Classification Report:")
print(classification_report(merged_df_svm['Genre'], merged_df_svm['Predicted_Genre_SVM'],target_names=label_encoder.classes_))



SVM Accuracy: 0.5921771217712177
SVM Classification Report:
              precision    recall  f1-score   support

      action       0.43      0.34      0.38      1314
       adult       0.58      0.35      0.43       590
   adventure       0.52      0.21      0.30       775
   animation       0.46      0.13      0.20       498
   biography       0.00      0.00      0.00       264
      comedy       0.53      0.59      0.56      7446
       crime       0.30      0.04      0.07       505
 documentary       0.69      0.85      0.76     13096
       drama       0.55      0.76      0.64     13612
      family       0.49      0.10      0.17       783
     fantasy       0.44      0.07      0.13       322
   game-show       0.82      0.60      0.69       193
     history       0.00      0.00      0.00       243
      horror       0.65      0.58      0.61      2204
       music       0.64      0.50      0.56       731
     musical       0.38      0.04      0.07       276
     mystery       0.

In [29]:
zoner_Description = [
    'Explosive fight scenes in the city streets',
    'A haunted mansion that traps its visitors',
    'A brave adventurer in search of lost treasure',
    'A forbidden romance in the 1920s',
    'A daring rescue mission with a love interest'

    ]

test_data_tfidf = vectorizer.transform(zoner_Description)


y_pred_lr = linear_model.predict(test_data_tfidf)
predicted_genres_lr = label_encoder.inverse_transform(y_pred_lr)

y_pred_nb = nb_model.predict(test_data_tfidf)
predicted_genres_nb = label_encoder.inverse_transform(y_pred_nb)

y_pred_svm = svm_model.predict(test_data_tfidf)
predicted_genres_svm = label_encoder.inverse_transform(y_pred_svm)


print("Predicted Genres using Logistic Regression :", predicted_genres_lr)
print("Predicted Genres using Naive Bayes :", predicted_genres_nb)
print("Predicted Genres using SVM :", predicted_genres_svm)
print()
for i,message in enumerate(zoner_Description):
  print(f"Story : {message}")
  print(f"Status :\tNaive Bayes Prediction :{predicted_genres_nb[i]}")
  print(f"\t\tLogistic Regression Prediction :{predicted_genres_lr[i]}")
  print(f"\t\tSVM Prediction :{predicted_genres_svm[i]}")
  print("="*100)








Predicted Genres using Logistic Regression : ['documentary' 'horror' 'adventure' 'drama' 'drama']
Predicted Genres using Naive Bayes : ['documentary' 'horror' 'documentary' 'drama' 'drama']
Predicted Genres using SVM : ['documentary' 'horror' 'adventure' 'drama' 'comedy']

Story : Explosive fight scenes in the city streets
Status :	Naive Bayes Prediction :documentary
		Logistic Regression Prediction :documentary
		SVM Prediction :documentary
Story : A haunted mansion that traps its visitors
Status :	Naive Bayes Prediction :horror
		Logistic Regression Prediction :horror
		SVM Prediction :horror
Story : A brave adventurer in search of lost treasure
Status :	Naive Bayes Prediction :documentary
		Logistic Regression Prediction :adventure
		SVM Prediction :adventure
Story : A forbidden romance in the 1920s
Status :	Naive Bayes Prediction :drama
		Logistic Regression Prediction :drama
		SVM Prediction :drama
Story : A daring rescue mission with a love interest
Status :	Naive Bayes Predictio